In [1]:
from typing import Union, List, Dict

from utils import logger
logger.fmt_string = "%(name)s: %(message)s"

from backtesting.data import OrderStatus, OrderRequest
from backtesting.strategy import Strategy
from utils.data import Trade, OrderBook
from backtesting import readers, backtest
from backtesting.output import StorageOutput
from backtesting.readers import OrderbookReader

from metrics.filters import Filters
from metrics.metrics import *

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
class GatlingMM(Strategy):
  def __init__(self, side_volume=30000):
    super().__init__()
    self.side_volume = side_volume
    self.volumes_left = {} # add cancellation policy

  def define_orders(self, row: Union[Trade, OrderBook],
                    statuses: List[OrderStatus],
                    memory: Dict[str, Union[Trade, OrderBook]]) -> List[OrderRequest]:
    if self.balance.get(row.symbol, None) is None:
      if isinstance(row, OrderBook) and self.balance.get(row.symbol, None) is None:
        # Initialize first orders
        ask_volume = min(self._get_allowed_volume(row.symbol, memory, 'ask'), self.side_volume)
        self.volumes_left[(row.symbol, 'ask')] = self.side_volume - ask_volume
        ask_order = OrderRequest.create_ask(row.ask_prices[0], ask_volume, row.symbol, row.timestamp)

        bid_volume = min(self._get_allowed_volume(row.symbol, memory, 'bid'), self.side_volume)
        self.volumes_left[(row.symbol, 'bid')] = self.side_volume - bid_volume
        bid_order = OrderRequest.create_bid(row.bid_prices[0], bid_volume, row.symbol, row.timestamp)

        return [ask_order, bid_order]
    elif self.balance.get(row.symbol, None) is not None:
      orders = []
      for status in statuses:
        order: OrderRequest = self.active_orders[status.id]

        if status.status != 'partial': # finished and cancel
          self.volumes_left[(order.symbol, order.side)] += order.volume - order.volume_filled
        elif status.status == 'partial':
          self.volumes_left[(order.symbol, order.side)] += status.volume

      for (symbol, side), left_volume in self.volumes_left.items():
        if left_volume > 500:
          volume = min(left_volume, self._get_allowed_volume(symbol, memory, side))
          self.volumes_left[(symbol, side)] -= volume

          if side == 'bid':
            price = memory[('orderbook', symbol)].bid_prices[0]
          else:
            price = memory[('orderbook', symbol)].ask_prices[0]

          neworder = OrderRequest.create(price, volume, symbol, side, row.timestamp)
          orders.append(neworder)

      return orders
    return []

In [3]:
reader = OrderbookReader('../tests/resources/huge_dataset/orderbook_10_03_20.csv.gz', 
                         '../tests/resources/huge_dataset/trades_10_03_20.csv.gz',
                         stop_after=300000, depth_to_load=3)

output = StorageOutput([], [])
output.balances = []
strategy = GatlingMM()
strategy.balance_listener = lambda b, t: output.balances.append((t, b))

backtester = backtest.Backtest(reader, strategy, delay=300)


<backtest>: Initialized <Backtest with reader=<orderbook-reader on orderbook_file=../tests/resources/huge_dataset/orderbook_10_03_20.csv.gz, trades_file=../tests/resources/huge_dataset/trades_10_03_20.csv.gz, batch_nrows=10000>>


In [4]:
backtester.run()

<backtest>: Backtest initialize run
<Strategy>: New order: <order-request id=0, command=new, symbol=XBTUSD, side=ask, volume=30000, price=7823.0, volume_filled=0>
<Strategy>: New order: <order-request id=1, command=new, symbol=XBTUSD, side=bid, volume=30000, price=7822.5, volume_filled=0>
<Strategy>: New order: <order-request id=2, command=new, symbol=ETHUSD, side=ask, volume=30000, price=199.05, volume_filled=0>
<Strategy>: New order: <order-request id=3, command=new, symbol=ETHUSD, side=bid, volume=12138, price=199.0, volume_filled=0>
<Strategy>: New order: <order-request id=4, command=new, symbol=ETHUSD, side=bid, volume=12138, price=199.0, volume_filled=0>
<Strategy>: New order: <order-request id=5, command=new, symbol=ETHUSD, side=bid, volume=5724, price=199.0, volume_filled=0>
<Strategy>: Received status: <order-status id=2, status=cancel, volume_total=-1, at=2020-03-10 18:25:15.142000>
<Strategy>: New order: <order-request id=6, command=new, symbol=ETHUSD, side=ask, volume=30000

<Strategy>: Received status: <order-status id=15, status=partial, volume_total=4259, at=2020-03-10 18:25:37.255000>
<Strategy>: Received status: <order-status id=10, status=partial, volume_total=6424, at=2020-03-10 18:25:37.255000>
<Strategy>: Received status: <order-status id=15, status=partial, volume_total=4289, at=2020-03-10 18:25:37.255000>
<Strategy>: Received status: <order-status id=10, status=partial, volume_total=6454, at=2020-03-10 18:25:37.255000>
<Strategy>: Received status: <order-status id=15, status=partial, volume_total=4539, at=2020-03-10 18:25:37.255000>
<Strategy>: Received status: <order-status id=10, status=partial, volume_total=6704, at=2020-03-10 18:25:37.255000>
<Strategy>: Received status: <order-status id=15, status=partial, volume_total=4619, at=2020-03-10 18:25:37.255000>
<Strategy>: Received status: <order-status id=10, status=partial, volume_total=6784, at=2020-03-10 18:25:37.255000>
<Strategy>: Received status: <order-status id=15, status=finished, volum

<Strategy>: Received status: <order-status id=26, status=partial, volume_total=953, at=2020-03-10 18:25:41.699000>
<Strategy>: Received status: <order-status id=12, status=partial, volume_total=166, at=2020-03-10 18:25:41.863000>
<Strategy>: Received status: <order-status id=26, status=partial, volume_total=1293, at=2020-03-10 18:25:41.888000>
<Strategy>: New order: <order-request id=31, command=new, symbol=XBTUSD, side=bid, volume=544.0, price=7815.0, volume_filled=0>
<Strategy>: Received status: <order-status id=12, status=partial, volume_total=243, at=2020-03-10 18:25:41.899000>
<Strategy>: Received status: <order-status id=26, status=partial, volume_total=5293, at=2020-03-10 18:25:42.005000>
<Strategy>: New order: <order-request id=32, command=new, symbol=XBTUSD, side=bid, volume=4000.0, price=7815.0, volume_filled=0>
<Strategy>: Received status: <order-status id=26, status=partial, volume_total=7793, at=2020-03-10 18:25:42.109000>
<Strategy>: New order: <order-request id=33, comma

<Strategy>: New order: <order-request id=46, command=new, symbol=XBTUSD, side=bid, volume=1177.0, price=7814.0, volume_filled=0>
<Strategy>: Received status: <order-status id=42, status=partial, volume_total=3328, at=2020-03-10 18:25:59.025000>
<Strategy>: Received status: <order-status id=38, status=partial, volume_total=264, at=2020-03-10 18:25:59.025000>
<Strategy>: Received status: <order-status id=42, status=partial, volume_total=3373, at=2020-03-10 18:25:59.025000>
<Strategy>: Received status: <order-status id=38, status=partial, volume_total=308, at=2020-03-10 18:25:59.025000>
<Strategy>: Received status: <order-status id=42, status=partial, volume_total=3403, at=2020-03-10 18:25:59.025000>
<Strategy>: Received status: <order-status id=38, status=partial, volume_total=338, at=2020-03-10 18:25:59.025000>
<Strategy>: Received status: <order-status id=42, status=partial, volume_total=5903, at=2020-03-10 18:25:59.025000>
<Strategy>: Received status: <order-status id=38, status=parti

<Strategy>: New order: <order-request id=62, command=new, symbol=ETHUSD, side=ask, volume=2284.0, price=198.85, volume_filled=0>
<Strategy>: Received status: <order-status id=34, status=partial, volume_total=13095, at=2020-03-10 18:27:02.788000>
<Strategy>: Received status: <order-status id=34, status=partial, volume_total=13125, at=2020-03-10 18:27:02.788000>
<Strategy>: Received status: <order-status id=34, status=partial, volume_total=13187, at=2020-03-10 18:27:02.788000>
<Strategy>: Received status: <order-status id=34, status=partial, volume_total=13247, at=2020-03-10 18:27:02.788000>
<Strategy>: New order: <order-request id=63, command=new, symbol=ETHUSD, side=ask, volume=1074.0, price=198.9, volume_filled=0>
<Strategy>: Received status: <order-status id=34, status=partial, volume_total=13262, at=2020-03-10 18:27:10.686000>
<Strategy>: Received status: <order-status id=34, status=partial, volume_total=13282, at=2020-03-10 18:27:10.686000>
<Strategy>: Received status: <order-statu

<Strategy>: Received status: <order-status id=44, status=partial, volume_total=28696, at=2020-03-10 18:27:49.605000>
<Strategy>: New order: <order-request id=75, command=new, symbol=XBTUSD, side=ask, volume=12424, price=7828.5, volume_filled=0>
<Strategy>: New order: <order-request id=76, command=new, symbol=XBTUSD, side=ask, volume=7576.0, price=7828.5, volume_filled=0>
<Strategy>: Received status: <order-status id=34, status=partial, volume_total=19389, at=2020-03-10 18:27:49.639000>
<Strategy>: Received status: <order-status id=65, status=partial, volume_total=130, at=2020-03-10 18:27:49.639000>
<Strategy>: Received status: <order-status id=34, status=partial, volume_total=19409, at=2020-03-10 18:27:49.639000>
<Strategy>: Received status: <order-status id=65, status=partial, volume_total=150, at=2020-03-10 18:27:49.639000>
<Strategy>: Received status: <order-status id=34, status=partial, volume_total=19886, at=2020-03-10 18:27:49.639000>
<Strategy>: Received status: <order-status id

<Strategy>: Received status: <order-status id=86, status=partial, volume_total=614, at=2020-03-10 18:27:53.419000>
<Strategy>: Received status: <order-status id=81, status=finished, volume_total=-1, at=2020-03-10 18:27:53.424000>
<Strategy>: Received status: <order-status id=82, status=partial, volume_total=5, at=2020-03-10 18:27:53.424000>
<Strategy>: Received status: <order-status id=86, status=partial, volume_total=678, at=2020-03-10 18:27:53.424000>
<Strategy>: New order: <order-request id=89, command=new, symbol=ETHUSD, side=ask, volume=1558.0, price=199.15, volume_filled=0>
<Strategy>: Received status: <order-status id=55, status=finished, volume_total=-1, at=2020-03-10 18:27:53.872000>
<Strategy>: Received status: <order-status id=82, status=finished, volume_total=-1, at=2020-03-10 18:27:53.872000>
<Strategy>: Received status: <order-status id=83, status=finished, volume_total=-1, at=2020-03-10 18:27:53.872000>
<Strategy>: Received status: <order-status id=86, status=finished, v

<Strategy>: New order: <order-request id=103, command=new, symbol=XBTUSD, side=bid, volume=15175, price=7826.0, volume_filled=0>
<Strategy>: New order: <order-request id=104, command=new, symbol=XBTUSD, side=bid, volume=14825.0, price=7826.0, volume_filled=0>
<Strategy>: Received status: <order-status id=103, status=partial, volume_total=15000, at=2020-03-10 18:29:07.425000>
<Strategy>: Received status: <order-status id=103, status=finished, volume_total=-1, at=2020-03-10 18:29:07.425000>
<Strategy>: Received status: <order-status id=104, status=partial, volume_total=1081, at=2020-03-10 18:29:07.425000>
<Strategy>: New order: <order-request id=105, command=new, symbol=XBTUSD, side=bid, volume=10000, price=7826.0, volume_filled=0>
<Strategy>: New order: <order-request id=106, command=new, symbol=XBTUSD, side=bid, volume=6256.0, price=7826.0, volume_filled=0>
<Strategy>: Received status: <order-status id=104, status=partial, volume_total=1112, at=2020-03-10 18:29:07.443000>
<Strategy>: R

KeyboardInterrupt: 

In [ ]:
strategy.balance

In [9]:
backtester.memory[('orderbook', 'XBTUSD')].bid_prices[0] * strategy.balance['XBTUSD'] + \
    backtester.memory[('orderbook', 'ETHUSD')].bid_prices[0] * strategy.balance['ETHUSD'] + \
    strategy.balance['USD']

431530.0124215501

In [10]:
backtester.memory[('orderbook', 'XBTUSD')].bid_prices[0], backtester.memory[('orderbook', 'XBTUSD')].ask_prices[0]

(7829.0, 7829.5)